In [22]:
import ROOT
from ROOT import TMath
import time

In [23]:
#Higgs
f_gghzzllll = ROOT.TFile.Open("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/4lep/MC/mc_345060.ggH125_ZZ4lep.4lep.root") 
f_whzzllll = ROOT.TFile.Open("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/4lep/MC/mc_341964.WH125_ZZ4lep.4lep.root")
f_vbfhzzllll = ROOT.TFile.Open("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/4lep/MC/mc_344235.VBFH125_ZZ4lep.4lep.root") 
f_zhzzllll = ROOT.TFile.Open("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/4lep/MC/mc_341947.ZH125_ZZ4lep.4lep.root")

#ZZ
f_zzllll = ROOT.TFile.Open("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/4lep/MC/mc_363490.llll.4lep.root")
f_zzllvv = ROOT.TFile.Open("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/4lep/MC/mc_363492.llvv.4lep.root")
f_zzllqq = ROOT.TFile.Open("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/4lep/MC/mc_363356.ZqqZll.4lep.root")

#Other
f_lllv = ROOT.TFile.Open("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/4lep/MC/mc_363491.lllv.4lep.root")
f_zee = ROOT.TFile.Open("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/4lep/MC/mc_361106.Zee.4lep.root")
f_zmumu = ROOT.TFile.Open("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/4lep/MC/mc_361107.Zmumu.4lep.root")
f_ztautau = ROOT.TFile.Open("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/4lep/MC/mc_361108.Ztautau.4lep.root")
f_ttllll = ROOT.TFile.Open("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/4lep/MC/mc_410000.ttbar_lep.4lep.root")

#Data
f_data_A = ROOT.TFile.Open("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/4lep/Data/data_A.4lep.root")
f_data_B = ROOT.TFile.Open("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/4lep/Data/data_B.4lep.root")
f_data_C = ROOT.TFile.Open("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/4lep/Data/data_C.4lep.root")
f_data_D = ROOT.TFile.Open("https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/4lep/Data/data_D.4lep.root")

files = {
    "data_A": f_data_A,  
    "data_B": f_data_B,
    "data_C": f_data_C,
    "data_D": f_data_D,
    "ggH_ZZ_llll": f_gghzzllll,
    "WH_ZZ_llll": f_whzzllll,
    "ZH_ZZ_llll": f_zhzzllll,
    "VBFH_ZZ_llll": f_vbfhzzllll,
    "ZZllll": f_zzllll,
    "ZZllvv": f_zzllvv,
    "ZZllqq": f_zzllqq,
    "lllv:": f_lllv,
    "Zee": f_zee,
    "Zmumu": f_zee,
    "Ztautau": f_ztautau,
    "ttllll": f_ttllll
}

In [24]:
%jsroot on

In [25]:
histos = {}
#for f in files:
#    if(("data" not in f) and ("H" not in f) ):
#        histos[f] = ROOT.TH1F("","",24,80,170)
histos["Data"] = ROOT.TH1F("","",24,80,170)
histos["Higgs"] = ROOT.TH1F("","",24,80,170)
histos["ZZ"] = ROOT.TH1F("","",24,80,170)
histos["Other"] = ROOT.TH1F("","",24,80,170)

In [26]:
print("Total number of samples = " + str(len(files)))
weight = 1.0

lumi = 10000.0

frac = 1.0

for f in files:
    tree = files[f].Get("mini")

    print("samples = "+ str(f) + " with " + str(tree.GetEntries()))
    
    Lepton_1 = ROOT.TLorentzVector()
    Lepton_2 = ROOT.TLorentzVector()

    zCand = ROOT.TLorentzVector()
    hCand = ROOT.TLorentzVector()

    n = 0
    #for event in tree:
    #maxEvents = 20000
    if ("data" in f):
        maxEvents = tree.GetEntries()
    else:
        maxEvents = int(tree.GetEntries()*frac)

    for event in range(0,maxEvents):
        tree.GetEvent(event)
        zCands = []
        hCands = []
        rawLeps = []
        
        if((tree.trigM==0) and (tree.trigE==0)):
            continue
        
        if ("data" in f):
            weight = 1.0
        else:
            #print("cross section = " + str(tree.XSection))
            effLumi = (tree.SumWeights*frac)/(tree.XSection)
            sf  = tree.scaleFactor_PILEUP*tree.scaleFactor_LepTRIGGER
            weight = (tree.mcWeight)*(lumi/effLumi)

        n += 1
        ## printing the evolution in number of events
        if(n%20000==0):
            print(str(n) + " / " + str(tree.GetEntries())) 

        if(len(tree.lep_phi) < 4):
            continue
            
        for j in range(0, len(tree.lep_phi)):
            rawLepton = ROOT.TLorentzVector()
            rawLepton.SetPtEtaPhiE(tree.lep_pt[j]/1000., tree.lep_eta[j], tree.lep_phi[j],tree.lep_E[j]/1000.)
            rawLeps.append(rawLepton)
        rawLepsSorted = sorted(rawLeps, key=lambda obj: obj.Pt(), reverse=True)
        #print("pts:  " + str(rawLepsSorted[0].Pt()) + " " + str(rawLepsSorted[1].Pt()) + " " + str(rawLepsSorted[2].Pt()) + " " + str(rawLepsSorted[3].Pt())  )

        if ((rawLepsSorted[0].Pt() < 25.0) or (rawLepsSorted[1].Pt() < 15.0) or (rawLepsSorted[2].Pt() < 10.0) or (rawLepsSorted[3].Pt() < 7.0)):
            continue
        
        #sumCharge = 0.0
        #for j in range(0, len(tree.lep_phi)):
        #    sumCharge += tree.lep_charge[j]
        
        #if sumCharge !=0:
        #    continue
            
        for j in range(0, len(tree.lep_phi)):
            #print("pt iso = " + str(tree.lep_ptcone30[j] / tree.lep_pt[j]) + " et iso =  " + str(tree.lep_etcone20[j] / tree.lep_pt[j]))
            Lepton_1.SetPtEtaPhiE(tree.lep_pt[j]/1000., tree.lep_eta[j], tree.lep_phi[j],tree.lep_E[j]/1000.)
            for k in range(0, len(tree.lep_phi)):  
                Lepton_2.SetPtEtaPhiE(tree.lep_pt[k]/1000., tree.lep_eta[k], tree.lep_phi[k],tree.lep_E[k]/1000.)
                if((tree.lep_pt[j] > 7000) and (TMath.Abs(tree.lep_eta[j]) < 2.5) and (tree.lep_ptcone30[j] / tree.lep_pt[j] < 0.3) and (tree.lep_etcone20[j] / tree.lep_pt[j] < 0.3) and (tree.lep_charge[j] == -1)  and (TMath.Abs(tree.lep_trackd0pvunbiased[j]/tree.lep_tracksigd0pvunbiased[j]) < 5) and (TMath.Abs(tree.lep_z0[j]*TMath.Sin(Lepton_1.Theta())) < 0.5)  ):
                    if((tree.lep_pt[k] > 7000) and (TMath.Abs(tree.lep_eta[k]) < 2.5) and (tree.lep_ptcone30[k] / tree.lep_pt[k] < 0.3) and (tree.lep_etcone20[k] / tree.lep_pt[k] < 0.3) and (tree.lep_charge[k] == 1) and (TMath.Abs(tree.lep_trackd0pvunbiased[k]/tree.lep_tracksigd0pvunbiased[k]) < 5) and (TMath.Abs(tree.lep_z0[k]*TMath.Sin(Lepton_2.Theta())) < 0.5) ):                
                        if (TMath.Abs(tree.lep_type[j]) == TMath.Abs(tree.lep_type[k])):
                            zCand = Lepton_1 + Lepton_2
                            zCands.append(zCand)

        mZ = 91.18
        zCandsSorted = sorted(zCands, key=lambda obj: abs(mZ - obj.M()))
        
        if (len(zCands) > 1):
            hCand = zCandsSorted[0] + zCandsSorted[1]
            hCands.append(hCand)
        

        for hCand in hCands:
            if("data" in f):
                histos["Data"].Fill(hCand.M(), weight)
            elif("H" in f):
                histos["Higgs"].Fill(hCand.M(), weight)
            elif("ZZ" in f):
                histos["ZZ"].Fill(hCand.M(), weight)
            else:
                histos["Other"].Fill(hCand.M(), weight)


Total number of samples = 16
samples = data_C with 237
samples = data_B with 156
samples = data_A with 39
samples = ZZllll with 554279
20000 / 554279
40000 / 554279
60000 / 554279
80000 / 554279
100000 / 554279
120000 / 554279
140000 / 554279
160000 / 554279
180000 / 554279
200000 / 554279
220000 / 554279
280000 / 554279
300000 / 554279
320000 / 554279
340000 / 554279
360000 / 554279
380000 / 554279
400000 / 554279
420000 / 554279
440000 / 554279
460000 / 554279
480000 / 554279
500000 / 554279
520000 / 554279
540000 / 554279
samples = data_D with 400
samples = ggH_ZZ_llll with 164716
20000 / 164716
40000 / 164716
60000 / 164716
80000 / 164716
100000 / 164716
120000 / 164716
140000 / 164716
160000 / 164716
samples = ZH_ZZ_llll with 14485
samples = Ztautau with 9
samples = Zee with 898
samples = WH_ZZ_llll with 15379
samples = VBFH_ZZ_llll with 191126
20000 / 191126
40000 / 191126
60000 / 191126
80000 / 191126
100000 / 191126
120000 / 191126
140000 / 191126
160000 / 191126
180000 / 19112

In [27]:
canvas = ROOT.TCanvas("Canvas","cz",800,600)
stack = ROOT.THStack()

colour = 2

for h in histos:
    print(h)
    histos[h].SetStats(0)
    if ("Data" not in h):
        histos[h].SetFillColor(colour)
        #stack.Add(histos[h])
        colour = colour + 1

stack.Add(histos["Other"])
stack.Add(histos["ZZ"])
stack.Add(histos["Higgs"])

stack.SetMaximum(35.0)
stack.Draw("hist")
histos["Data"].Draw("E1psame")


stack.GetXaxis().SetTitle("m_{llll} [GeV]")
stack.GetYaxis().SetTitle("Events")

stack.GetXaxis().SetTitleOffset(1.2)
stack.GetXaxis().SetTitleSize(23.0)

leg = ROOT.TLegend(0.6,0.6,0.85,0.85)
for h in histos:
    if("Data" not in h):
        leg.AddEntry(histos[h],h, "f")
    else:
        leg.AddEntry(histos[h], h, "l")

leg.Draw()
canvas.Draw()

Other
Data
ZZ
Higgs


Warning in <TCanvas::Constructor>: Deleting canvas with same name: Canvas


In [ ]:
f = ROOT.TFile("results.root","RECREATE")
canvas.Write()
stack.Write()
for h in histos:
    histos[h].Write(h)
    
f.Write()
f.Close()
           
print("files written")
           

In [36]:
#stack.RecursiveRemove(histos["Higgs"])
#sum_bonly = stack.GetStack().Last()

f = ROOT.TFile("results.root","RECREATE")
h_data = ROOT.TH1F(f.Get("Data"))
h_zz = ROOT.TH1F(f.Get("ZZ"))
h_other = ROOT.TH1F(f.Get("Other"))

for bin in range(1,h_zz.GetNbinsX()):
    total_bkg = h_zz.GetBonContent(bin) + h_other.GetBonContent(bin)
    print("bin content = " + str(total_bkg)  )

TypeError: none of the 6 overloaded methods succeeded. Full details:
  TH1F::TH1F() =>
    takes at most 0 arguments (1 given)
  TH1F::TH1F(const TVectorT<float>& v) =>
    could not convert argument 1
  TH1F::TH1F(const TH1F& h1f) =>
    could not convert argument 1
  TH1F::TH1F(const char* name, const char* title, int nbinsx, double xlow, double xup) =>
    takes at least 5 arguments (1 given)
  TH1F::TH1F(const char* name, const char* title, int nbinsx, const float* xbins) =>
    takes at least 4 arguments (1 given)
  TH1F::TH1F(const char* name, const char* title, int nbinsx, const double* xbins) =>
    takes at least 4 arguments (1 given)

In [ ]:
canvas.Draw()